# Data Preperation for TESSA evaluation

Here, we perpare our data to the format of TESSA from:

Zhang Z, Xiong D, Wang X, Liu H, Wang T. Mapping the functional landscape of T cell receptor repertoires by single-T cell transcriptomics. Nat Methods. 2021.

https://www.nature.com/articles/s41592-020-01020-3

Clone the Github repository from https://github.com/jcao89757/TESSA to the folder 'baseline'.

In [1]:
import scanpy as sc
import pandas as pd
import os
import sys

sys.path.append('..')
import tcr_embedding as tcr
from tcr_embedding.utils_preprocessing import group_shuffle_split
import config.constants_10x as const

# Transform data

In [2]:
path_10x_adata = '../data/10x_CD8TC/v6_supervised.h5ad'
adata = sc.read_h5ad(path_10x_adata)

adatas_per_donor = []
for i in range(1, 3):
    new_ad = adata[adata.obs['donor']== f'donor_{i}']
    new_ad.obs['binding_name'] = new_ad.obs['binding_name'].astype(str)
    new_ad = new_ad[(new_ad.obs['binding_name'].isin(const.HIGH_COUNT_ANTIGENS))]
    adatas_per_donor.append(new_ad.copy())

C:\Users\felix.drost\Anaconda3\envs\mvTCR\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


In [3]:
splits = {}

for j, adata_donor in enumerate(adatas_per_donor):
    splits[f'donor_{j+1}'] = {}
    adata_donor.obs['group_col'] = [seq[1:-1] for seq in adata_donor.obs['IR_VDJ_1_cdr3']]
    for i in range(0, 5):
        random_seed = i
        adata_tmp = adata_donor.copy()
        train_val, test = group_shuffle_split(adata_tmp, group_col='group_col', val_split=0.20, random_seed=random_seed)
        train, val = group_shuffle_split(train_val, group_col='group_col', val_split=0.25, random_seed=random_seed)
        
        adata_tmp.obs['set'] = 'train'
        adata_tmp.obs.loc[val.obs.index, 'set'] = 'val'
        adata_tmp.obs.loc[test.obs.index, 'set'] = 'test'
        
        adata_tmp = adata_tmp[adata_tmp.obs['set'].isin(['train', 'test'])]
        splits[f'donor_{j+1}'][f'split_{i}'] = adata_tmp.copy()

C:\Users\felix.drost\Anaconda3\envs\mvTCR\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
C:\Users\felix.drost\Anaconda3\envs\mvTCR\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
C:\Users\felix.drost\Anaconda3\envs\mvTCR\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
C:\Users\felix.drost\Anaconda3\envs\mvTCR\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_catego

Write the TCR sequences to seperate csv file for each donor:

In [4]:
for donor, ad_dict in splits.items():
    for split, adata in ad_dict.items():
        path_base = f'../data/tessa/10x/{donor}/'
        if not os.path.exists(path_base):
            os.mkdir(path_base)

        df_tcr = adata.obs[['IR_VDJ_1_cdr3', 'binding_name', 'set']]
        df_tcr['cdr3'] = [seq[1:-1] for seq in df_tcr['IR_VDJ_1_cdr3']]

        df_tcr = df_tcr[['cdr3', 'binding_name', 'set']]
        df_tcr.index.name = 'contig_id'

        df_tcr_train = df_tcr[df_tcr['set']=='train']
        df_tcr_test = df_tcr[df_tcr['set']=='test']

        df_tcr_train.to_csv(path_base+f'{split}_tcrs_atlas.csv')
        df_tcr_test.to_csv(path_base+f'{split}_tcrs_query.csv')

C:\Users\FELIX~1.DRO\AppData\Local\Temp/ipykernel_7332/4272327156.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tcr['cdr3'] = [seq[1:-1] for seq in df_tcr['IR_VDJ_1_cdr3']]


Write the scRNA matrix to seperate csv file for each donor:

In [5]:
for donor, ad_dict in splits.items():
    for split, adata in ad_dict.items():
        path_base = f'../data/tessa/10x/{donor}/'
        count_mat = adata.X.A
        df_counts = pd.DataFrame(count_mat)

        df_counts.index = adata.obs.index
        df_counts.index.name = ''
        df_counts.columns = adata.var.index

        df_counts_train = df_counts[adata.obs['set']=='train']
        df_counts_test = df_counts[adata.obs['set']=='test']


        df_counts_train = df_counts_train.transpose()
        df_counts_test = df_counts_test.transpose()

        df_counts_train.to_csv(path_base+f'{split}_scRNA_atlas.csv')
        df_counts_test.to_csv(path_base+f'{split}_scRNA_query.csv')

## Transform the Covid Dataset

In [6]:
def extract_reactivity(value):
    if 'un-reactive' in value:
        return 'un-reactive'
    if 'CD8' in value:
        return 'CD8 reactive'
    if 'CD4' in value:
        return 'CD4 reactive'

In [7]:
path_covid_adata = '../data/Covid/04_covid_highly_var_5000.h5ad'
adata = sc.read_h5ad(path_covid_adata)
adata.obs['reactivity'] = [extract_reactivity(x) for x in adata.obs['reactive_combined']]

In [8]:
splits_covid = {}

for i in range(0, 5):
    random_seed = i
    adata_tmp = adata.copy()
    sub, non_sub = group_shuffle_split(adata_tmp, group_col='TRB_1_cdr3', val_split=0.2, random_seed=random_seed)
    train, val = group_shuffle_split(sub, group_col='TRB_1_cdr3', val_split=0.20, random_seed=random_seed)
    adata_tmp.obs['set'] = 'train'
    adata_tmp.obs.loc[non_sub.obs.index, 'set'] = '-'
    adata_tmp.obs.loc[val.obs.index, 'set'] = 'val'
    adata_tmp = adata_tmp[adata_tmp.obs['set'].isin(['train', 'val'])]
    splits_covid[f'split_{i}'] = adata_tmp.copy()

C:\Users\felix.drost\Anaconda3\envs\mvTCR\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
C:\Users\felix.drost\Anaconda3\envs\mvTCR\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
C:\Users\felix.drost\Anaconda3\envs\mvTCR\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
C:\Users\felix.drost\Anaconda3\envs\mvTCR\lib\site-packages\pandas\core\arrays\categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_catego

In [9]:
path_base = f'../data/tessa/covid/'
if not os.path.exists(path_base):
    os.mkdir(path_base)

for split, adata in splits_covid.items():
    df_tcr = adata.obs[['TRB_1_cdr3', 'T_cells', 'reactivity' , 'set']]
    df_tcr['cdr3'] = [seq[1:-1] for seq in df_tcr['TRB_1_cdr3']]

    df_tcr = df_tcr[['cdr3', 'T_cells', 'reactivity', 'set']]
    df_tcr.index.name = 'contig_id'

    df_tcr.to_csv(path_base+f'{split}_tcrs_atlas.csv')

C:\Users\FELIX~1.DRO\AppData\Local\Temp/ipykernel_7332/2621316884.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tcr['cdr3'] = [seq[1:-1] for seq in df_tcr['TRB_1_cdr3']]


In [10]:
path_base = f'../data/tessa/covid/'

for split, adata in splits_covid.items():
    count_mat = adata.X.A
    df_counts = pd.DataFrame(count_mat)

    df_counts.index = adata.obs.index
    df_counts.index.name = ''
    df_counts.columns = adata.var.index

    df_counts = df_counts.transpose()
    df_counts.to_csv(path_base+f'{split}_scRNA_atlas.csv')